# Construcción de Técnicas de Normalización para Datos en Histograma time residual  vs cos(α)

In [5]:
from analysis_simulation_subset import Analysis_simulation, magnitude
import uproot
import numpy as np
from numpy import array, where, shape, reshape, pi

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import seaborn as sn

import pandas as pd

# 1 - Extraer datos

In [6]:
file = uproot.open("/snoplus simulations/electron bulk/center point/simu_Analysis_elec_10MeV.root")

data1 = file['T;6']
pmt_info = file['pmt;1']

In [7]:
file.keys()

['T;6', 'T;5', 'pmt;2', 'pmt;1']

## 1.1 - Datos de Simulación

#df = Analysis_simulation(file, [0,2,3,4])

df = Analysis_simulation(file)
df.head()

df.to_csv("elec_bulk_10MeV_centerpoint_randomdir.csv")

In [8]:
df = pd.read_csv("elec_bulk_10MeV_centerpoint_randomdir.csv")
df.head()

,Unnamed: 0,job,eventID,hitpmt ID,hit type,time residual,mc coordinates,ev direction,mc radius,reconst radius,reconst error,PMT xyz,PMT spherical
0,0,0,[ 0 0 0 ... 999 999 999],"[[5143, 9638, 6384, 5947, 4877, 2327, 2718, 59...","[[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[-136.57445262258054, -9.782284484595692, 3.4...","[array([0., 0., 0.]), array([0., 0., 0.]), arr...","[array([-2.51373434, 0.52063274, 10.17988491]...",[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[169.45076563 220.66149963 114.60298927 83.18...,[169.45076563 220.66149963 114.60298927 83.18...,"[[array([-5040.03, -3961.15, -5431.27]), array...","[[array([ 2.27370073e+00, -2.47548524e+00, 8...."
1,1,1,[ 0 0 0 ... 999 999 999],"[[7212, 7256, 5871, 7330, 7290, 8354, 337, 834...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[5.527437958958103, 3.0473798810866697, 2.420...","[array([0., 0., 0.]), array([0., 0., 0.]), arr...","[array([ 7.41879606, -6.78051758, -3.03414226]...",[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[205.38440546 174.36864098 157.69696893 224.42...,[205.38440546 174.36864098 157.69696893 224.42...,"[[array([ 1150.05, -5440.75, -6311.88]), array...","[[array([ 2.41935643e+00, -1.36248556e+00, 8...."
2,2,2,[ 0 0 0 ... 999 999 999],"[[4244, 4452, 5035, 5463, 4704, 4828, 5926, 42...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[109.32870277905525, 3.5975471309273814, 2.19...","[array([0., 0., 0.]), array([0., 0., 0.]), arr...","[array([-9.67355061, -3.43465281, -2.20126176]...",[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[232.21982312 83.84724827 190.78737072 172.84...,[232.21982312 83.84724827 190.78737072 172.84...,"[[array([-6993.87, 4359.12, -1738.54]), array...","[[array([1.77870689e+00, 2.58423314e+00, 8.422..."
3,3,3,[ 0 0 0 ... 999 999 999],"[[4499, 7306, 5620, 4182, 6608, 5689, 5677, 58...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[38.11211224922954, 2.942122472222934, 4.0101...","[array([0., 0., 0.]), array([0., 0., 0.]), arr...","[array([-7.95063114, -5.77963543, -3.68825912]...",[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[127.33553669 210.97553502 205.13882094 368.91...,[127.33553669 210.97553502 205.13882094 368.91...,"[[array([-8179.26, 1655.07, 1018.34]), array...","[[array([1.44936720e+00, 2.94193889e+00, 8.406..."


### 1.1.1 - Unify jobs info.

In [10]:
ev_direct = (df['ev direction']).to_numpy()
type(ev_direct)
ev_direct

array(['[array([-2.51373434,  0.52063274, 10.17988491]), array([-4.88648367,  7.86017466, -4.95579529]), array([-7.9698801 ,  3.50262165,  5.86793232]), array([-2.7524755 , -2.47218776,  9.82507706]), array([ -2.06262136, -10.27420521,   0.63739383]), array([ 3.77819991, -8.12554073,  5.46998787]), array([-10.15907097,  -1.97699165,   1.76203442]), array([10.35636616,  1.71852362,  0.11106868]), array([ 9.60795784, -1.80688059, -3.82652783]), array([-10.35177612,   1.0214783 ,  -1.4203155 ]), array([-5.64789391, -8.40140343, -2.78166986]), array([-2.0210371 ,  9.69658852,  3.48016524]), array([ 8.35917377, -3.17060947, -5.50376511]), array([-9.38726997, -2.15489531,  4.17798567]), array([5.47308731, 1.62840319, 8.80985832]), array([ 0.03847532, -9.1188488 ,  5.20241165]), array([-3.6214931 ,  0.41879961,  9.84527206]), array([-7.7703948 ,  5.57529449,  4.33093929]), array([-6.48272085,  0.42177477,  8.24720669]), array([-10.27719212,   1.69517517,  -1.3136543 ]), array([10.152174  , -1

In [40]:
ls = ['1']
ls[0]

'1'

In [11]:
#Info to be concatenated:

multi_time_residual = np.array([])
multi_pmt_ID = np.array([], dtype= np.int64)
multi_cos_angle = np.array([])
mc_momentum_multi_ev = []
pmt_multi_ev_xyz = []
pmt_multi_ev_sph = np.array([])


job = (df['job']).to_numpy()

#run over jobs
for job_i in job:
    print(job_i)
    
    job_n = df.loc[df['job'] == job_i]
    
    evtID_job = (job_n['eventID']).to_numpy()[0]
    time_residual_job = (job_n['time residual']).to_numpy()[0]        #Time residual structure --> time_res[0][evID]
    hitpmt_ID_job = (job_n['hitpmt ID']).to_numpy()[0]                #hitpmt_ID structure --> hitpmt_ID[0][evID]
    mc_momentum = (job_n['ev direction']).to_numpy()[0]               # ev direction
    xyz_hit = (job_n['PMT xyz']).to_numpy()[0]                        #coord structure --> xyz_hit [0][evID]
    sph_hit = (job_n['PMT spherical']).to_numpy()[0]                  #coord structure --> sph_hit [0][evID]

    # On each job, run over the mcIDs and extract info for each ID
    for ID_i in np.unique(evtID_job):
        
        multi_time_residual = np.append(multi_time_residual, time_residual_job[ID_i]).ravel()
        multi_pmt_ID = np.append(multi_pmt_ID, hitpmt_ID_job[ID_i]).ravel()
        mc_momentum_multi_ev =np.append(mc_momentum_multi_ev, mc_momentum[ID_i])
        
        xyz_hit_id = xyz_hit[ID_i]
        sph_hit_id = sph_hit[ID_i]

        #compute cos(α)
        cos_id = []
        N = np.shape(xyz_hit_id)[0]
        vec_ev = np.array([0.0, 0.0, -1.0])
        for k in range(N):
            cos_val = np.dot(xyz_hit_id[k],vec_ev)/np.linalg.norm(xyz_hit_id[k])
            cos_id.append(cos_val)
        multi_cos_angle = np.append(multi_cos_angle, cos_id).ravel()


0


TypeError: string indices must be integers, not 'numpy.str_'

## 1.2 - PMT info

In [7]:
pmt_id = array(pmt_info['pmt_id'])
pmt_pos_xyz = array(pmt_info['pmt_pos_xyz'])
pmt_pos_sph = array(pmt_info['pmt_pos_sph'])
pmt_type = array(pmt_info['pmt_type'])

### 1.2.1 - Extraer los PMT type deseados (type = 1)

In [8]:
valid_type = 1
valid_pmt_id = []		            # valid PMTid
valid_pmt_xyz = []                  # xyz coord. of PMT type = 1 
valid_pmt_sph = []                  # sph coord. of PMT type = 1 

valid_info_pmt_i = where(pmt_type == valid_type)[0]

for i in valid_info_pmt_i:
    valid_pmt_id.append(pmt_id[i])
    valid_pmt_xyz.append(pmt_pos_xyz[i])
    valid_pmt_sph.append(pmt_pos_sph[i])

### 1.2.3 - Usar info. de PMTs para contabilizar el Nº de PMTs dentro de un subsección relacionada con el cos(α) - Dirección de evento

- 1ero. Calcular cos(α) para todos los PMTs
- 2ero. Dividir α en subsecciones (corresponder al binning del histograma). Usaremos el cos(α). α varia entre [0,π], y cos(α) entre [-1,1]
- 3do. Contabilizar el nº de PMTs dentro de está sección y guardar esta información -> Nº de PMTs = Nº de datos en cos(Δα)


In [9]:
#Compute cos(angle)
vec_ev = np.array([0.0, 0.0, -1.0])
cos_angle = np.array([])

N = shape(valid_pmt_xyz)[0] 
for i in range(N):
    cos_val = np.dot(valid_pmt_xyz[i],vec_ev)/np.linalg.norm(valid_pmt_xyz[i])
    cos_angle = np.append(cos_angle, cos_val)
cos_angle

array([ 0.76061486,  0.74355943,  0.72554996, ..., -0.1383896 ,
       -0.1664106 , -0.13705073])

In [16]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# Now devide the cos_angel in sections, lets use an Histogram and then extract the counts per bin
bins = 30

title = 'Nº of PMT in cos(Δα) section'
ytitle = 'PMT counts'
xtitle = 'cos(α)'

aval_PMT, x_, y_ = plt.hist(cos_angle, bins = bins, edgecolor='black', facecolor='g')
plt.ylabel(ytitle)
plt.xlabel(xtitle)
plt.title(title)
plt.show()

aval_PMT = aval_PMT.astype(int)

# 2 - Construcción de 2Dhist time_res vs. cos(Δα)


## 2.1 - Construción de histogramas a 2D multiEV

In [17]:
# Time residual cuts

up_cut = 14
inf_cut = -5

multi_time_res_cut = []
multi_cos_angle_cut = []
multi_pmt_ID_cut = []

for i in np.where((multi_time_residual > inf_cut) & (multi_time_residual < up_cut))[0]:

    multi_time_res_cut.append(multi_time_residual[i])
    multi_cos_angle_cut.append(multi_cos_angle[i])
    multi_pmt_ID_cut.append(multi_pmt_ID[i])


In [18]:
%matplotlib
# 2d Histogram
title = 'cos(α) - Multiple evID - 5 MeV'

#bins defined above when computing the Nº of PMTs in each Δα!

plt.figure(figsize=(8,8))
sn.set_style(rc = {'axes.facecolor': 'black'})
sn.histplot(x = multi_cos_angle_cut, y = multi_time_res_cut, bins = [bins, bins], stat='count', cbar = 'True', cmap = cm.nipy_spectral, hue_norm = (0,1))
plt.ylabel('time res.')
plt.xlabel('cos(α)')
plt.title(title)

#equal acis ration
#ax = plt.gca()
#ax.set_aspect('equal', adjustable='box')
plt.show()

Using matplotlib backend: TkAgg


## 2.1 - Normalization

First, extract the count to be Normalized for the 2dHistogram

In [19]:
#Transforma Data into an Image Matrix
H, xedges, yedges = np.histogram2d(x = multi_cos_angle_cut, y = multi_time_res_cut, bins = [bins, bins])

#Now recover the data axis representation of time residual vs cos(alpha)
left = np.min(multi_time_res_cut)
right = np.max(multi_time_res_cut)
bottom = np.min(multi_cos_angle_cut)
top = np.max(multi_cos_angle_cut)

#Titles
title = 'cos(α) - Multiple evID - 5 MeV'
ylabel = 'cos(α)'
xlabel = 'time res.'

plt.imshow(H, extent = [left, right, top, bottom], aspect="auto" ) #---> Here is our image as a Matrix Pixel
plt.title(title)
plt.ylabel(ylabel)
plt.xlabel(xlabel)
plt.colorbar()

### 2.1.1 - Metodo 1: Dividir por el Nº de PMTs que dan hit en cada bin de cos(α)

In [21]:
# sum of hits in each bin = sum over each column element of the matrix.
sum_hit = np.sum(H,axis=0)  

H_normalized1 = H / sum_hit

N1title = 'Normalization by NHits in cos(α) section - Multiple evID - 5 MeV'

plt.imshow(H_normalized1, extent = [left, right, top, bottom], aspect="auto" )
plt.title(N1title)
plt.ylabel(ylabel)
plt.xlabel(xlabel)
plt.colorbar()

### 2.1.2 - Metodo 2: Dividir por el Nº de PMTs disponibles en cada bin de cos(α)

In [22]:
H_normalized2 = H / aval_PMT

N2title = 'Normalization by Nº PMTs in cos(α) section - Multiple evID - 5 MeV'

plt.imshow(H_normalized2, extent = [left, right, top, bottom], aspect="auto" )
plt.title(N2title)
plt.ylabel(ylabel)
plt.xlabel(xlabel)
plt.colorbar()